In [ ]:
#bruh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/[2024-2025]_AN2DL/Homework 1')

In [ ]:
#Create a requirements.txt file in which you'll specify the versions of the libraries you want

%%writefile requirements.txt
tensorflow==2.17.0
keras==3.4.1

In [ ]:
!pip install -r requirements.txt

In [ ]:
import numpy as np
import os

import tensorflow as tf
#from tensorflow import keras as tfk
import keras as tfk       #notice how I'm importing keras and not tensorflow.keras
from keras.layers import Input, Dense, Dropout, Lambda
#from tensorflow.keras.layers import Input, Dense, Dropout, Lambda
from keras import layers as tfkl


print(f"Tensorflow version (2.17) -> {tf.__version__}")
print(f"Keras version (3.4.1) -> {tfk.__version__}")

In [ ]:
!pip install keras-cv
!pip install tensorflow-probability


In [ ]:
print(f"Tensorflow version (2.17) -> {tf.__version__}")
print(f"Keras version (3.4.1) -> {tfk.__version__}")

In [ ]:
from keras import models, optimizers, callbacks
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
import matplotlib.pyplot as plt

from keras import mixed_precision
import keras_cv  # Keras CV for augmentation layers
#import tensorflow_probability as tfp
import random

In [ ]:
# Set mixed precision policy
#mixed_precision.set_global_policy('mixed_float16')#correct way to do it
#mixed_precision.set_global_policy('mixed_float16')
mixed_precision.set_global_policy('float32') #augmixdelcazzo

print(f"TensorFlow version -> {tf.__version__}")
print(f"Keras version -> {tfk.__version__}")
print(f"Keras CV version -> {keras_cv.__version__}")

In [ ]:
np.random.seed(42)
tf.random.set_seed(42);

##Data Import + train_val_test_split + balancing the data ♟

In [ ]:
#Import the cleaned dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

file_path = 'training_set_no_outliers.npz'

with np.load(file_path) as data:
    training_images = data['images']
    training_labels = data['labels']

#Check
print(f"Shape of images: {training_images.shape}")
print(f"type of images: {type(training_images)}")
print(f"shape of labels: {training_labels.shape}")
print(f"type of labels: {type(training_labels)}")
# Dataframe for labels
labels = training_labels.flatten()
df_labels = pd.DataFrame(labels, columns=['label'])
print(df_labels.head())

In [ ]:
# train test val split
from sklearn.model_selection import train_test_split
# Define split sizes
test_size = 0.2        # 20% for test
validation_size = 0.2  # 20% of the remaining 80% = 16% for validation
# Split into (training+validation) and test sets
training_labels = training_labels.flatten()
X_train_val, X_test, y_train_val, y_test = train_test_split(
    training_images,
    training_labels,
    test_size=test_size,
    random_state=42,
    stratify=training_labels
)
# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val,
    y_train_val,
    test_size=validation_size,
    random_state=42,
    stratify=y_train_val
)
print("Training Images - Min pixel value:", np.min(X_train))
print("Training Images - Max pixel value:", np.max(X_train))
print("Test Images - Min pixel value:", np.min(X_test))
print("Test Images - Max pixel value:", np.max(X_test))
print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Validation set shape: {X_val.shape}, {y_val.shape}")
print(f"Test set shape: {X_test.shape}, {y_test.shape}")

In [ ]:
# train val split only when you want to remove test set
from sklearn.model_selection import train_test_split
# Define split sizes
validation_size = 0.1  # 10%
# Split into (training+validation) and test sets
training_labels = training_labels.flatten()
X_train_val, X_val, y_train_val, y_val = train_test_split(
    training_images,
    training_labels,
    test_size=validation_size,
    random_state=42,
    stratify=training_labels
)

print("Training Images - Min pixel value:", np.min(X_train))
print("Training Images - Max pixel value:", np.max(X_train))
print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Validation set shape: {X_val.shape}, {y_val.shape}")


In [ ]:
#OVERSAMPLING
import numpy as np
class_counts = pd.Series(y_train).value_counts()
max_count = class_counts.max()
print(f"Maximum class count: {max_count}")
X_train_oversampled = []
y_train_oversampled = []
# Iterate through each class to perform oversampling
for class_label in class_counts.index:
    X_class = X_train[y_train == class_label]
    y_class = y_train[y_train == class_label]
    samples_needed = max_count - len(X_class)

    if samples_needed > 0:
        duplicates = samples_needed // len(X_class)
        remainder = samples_needed % len(X_class)
        for _ in range(duplicates):
            X_train_oversampled.append(X_class)
            y_train_oversampled.append(y_class)
        if remainder > 0:
            indices = np.random.choice(len(X_class), size=remainder, replace=True)
            X_train_oversampled.append(X_class[indices])
            y_train_oversampled.append(y_class[indices])
if X_train_oversampled:
    X_train_oversampled = np.vstack(X_train_oversampled)
    y_train_oversampled = np.hstack(y_train_oversampled)
    X_train_balanced = np.vstack((X_train, X_train_oversampled))
    y_train_balanced = np.hstack((y_train, y_train_oversampled))
else:
    X_train_balanced = X_train
    y_train_balanced = y_train
print(f"Training set shape after oversampling: {X_train_balanced.shape}, {y_train_balanced.shape}")

In [ ]:
# Count classes in training set
class_counts = pd.Series(y_train_balanced).value_counts().sort_index()
print("Class distribution in Training set after oversampling:")
print(class_counts)

# Plot for better visualization
plt.figure(figsize=(10,6))
sns.barplot(x=class_counts.index, y=class_counts.values, palette='viridis')
plt.title('Class Distribution in Training Set after Oversampling')
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.xticks(rotation=45)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_images_per_class(images, labels, classes, num=5):
    """
    Visualizza un numero specificato di immagini per ciascuna classe.

    Args:
    - images (numpy.ndarray): Array delle immagini.
    - labels (numpy.ndarray): Array delle etichette.
    - classes (list): Lista delle classi da visualizzare.
    - num (int): Numero di immagini per classe.
    """
    plt.figure(figsize=(15, len(classes) * 3))
    for i, cls in enumerate(classes):
        cls_indices = np.where(labels == cls)[0]
        # Controlla se ci sono abbastanza immagini per la classe
        if len(cls_indices) < num:
            print(f"Attenzione: La classe {cls} ha solo {len(cls_indices)} immagini. Verranno visualizzate tutte.")
            selected_indices = cls_indices
        else:
            selected_indices = np.random.choice(cls_indices, num, replace=False)
        for j, idx in enumerate(selected_indices):
            plt.subplot(len(classes), num, i*num + j + 1)
            plt.imshow(images[idx])
            plt.title(f"Classe: {cls}")
            plt.axis('off')
    plt.tight_layout()
    plt.show()

# Identificare tutte le classi
all_classes = class_counts.index.tolist()
print("Tutte le classi:", all_classes)

# Visualizzare 5 immagini per ciascuna delle 8 classi
plot_images_per_class(training_images, df_labels, all_classes, num=5)

##AUGMENT AND PREPARE FOR TRAINING + TESTING ⛹

In [ ]:
import tensorflow as tf
import keras as tfk  # Standalone Keras
from keras.layers import Input, Dense, Dropout, Lambda, GlobalAveragePooling2D
from keras import layers as tfkl
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.applications import InceptionResNetV2
import keras_cv
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
print(f"Tensorflow version (2.17) -> {tf.__version__}")
print(f"Keras version (3.4.1) -> {tfk.__version__}")

In [ ]:
input_shape = (224, 224, 3)
num_classes = 8  # Adjust based on your dataset
batch_size = 512  # You can adjust this based on your hardware

In [ ]:
random_flip_layer = keras_cv.layers.RandomFlip("horizontal_and_vertical")
random_rotation_layer = keras_cv.layers.RandomRotation(factor=0.4)
random_zoom_layer = keras_cv.layers.RandomZoom(height_factor=0.4, width_factor=0.4)
rand_augment_layer = keras_cv.layers.RandAugment(value_range=(0, 1), magnitude=0.4)# Applies random augmentations DESCREASED MAGNITUDE TO 0.3 HOPING IT WONT ET STUCK AT 0.95 (with just 0.3 gets stuck at 0.97), try at 0.2 AND 4 augmentations per image, got 084 instea of more tan 087

In [ ]:
random_flip_layer = keras_cv.layers.RandomFlip("horizontal_and_vertical")
random_rotation_layer= keras_cv.layers.RandomRotation(factor=0.2)
random_zoom_layer=keras_cv.layers.RandomZoom(height_factor=0.2, width_factor=0.2)
rand_augment_layer = keras_cv.layers.RandAugment(value_range=(0, 1), magnitude=0.2, augmentations_per_image =2)
random_noise_layer = tfk.layers.GaussianNoise(stddev=0.1)

In [ ]:
from keras.applications.inception_resnet_v2 import preprocess_input

In [ ]:
def preprocess_and_augment(images, labels):
    images = tf.image.resize(images, [224, 224])
    images = tf.cast(images, tf.float32) / 255.0
    images = random_flip_layer(images)
    images = random_rotation_layer(images)
    images = random_zoom_layer(images)
    images = rand_augment_layer(images)

    images = random_noise_layer(images)

    images = images * 255.0
    images = preprocess_input(images)
    labels = tf.one_hot(tf.cast(labels, tf.int32), depth=num_classes)
    return images, labels

def preprocess_val(images, labels):
    images = tf.image.resize(images, [224, 224])
    images = tf.cast(images, tf.float32)
    images = preprocess_input(images)
    labels = tf.one_hot(labels, depth=num_classes)
    labels = tf.cast(labels, tf.float32)
    return images, labels

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_balanced, y_train_balanced))
#debugging prints
for batch in train_dataset.take(1):
    print(f"Batch image shape: {batch[0].shape}")
    print(f"Batch label shape: {batch[1].shape}")
    break

# Shuffle and batch the dataset
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size)

# Apply augmentations on batched data
train_dataset = train_dataset.map(
    preprocess_and_augment,
    num_parallel_calls=tf.data.AUTOTUNE
)
#debugging prints
for batch in train_dataset.take(1):
    print(f"Batch image shape: {batch[0].shape}")
    print(f"Batch label shape: {batch[1].shape}")
    break

# Prefetch for performance
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

# Create TensorFlow Dataset from validation data
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))

# Apply preprocessing to validation dataset (no augmentations)
val_dataset = val_dataset.map(
    preprocess_val,
    num_parallel_calls=tf.data.AUTOTUNE
)

# Batch and prefetch validation dataset
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

#create TensorFlow Dataset from test data
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Apply preprocessing to test dataset (no augmentations)
test_dataset = test_dataset.map(
    preprocess_val,
    num_parallel_calls=tf.data.AUTOTUNE
)
# Batch and prefetch test dataset
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
def display_augmented_images(dataset, rows=3, cols=5):
    plt.figure(figsize=(15, 9))
    num_images = rows*cols
    count = 0
    for images, labels in dataset.take(1):  # Take one batch
        for i in range(num_images):
            if count >= num_images:
                break
            plt.subplot(rows, cols, count + 1)
            img = images[i].numpy()
            # Normalize values for visualization if they fall outside [0, 1]
            img = (img - np.min(img)) / (np.max(img) - np.min(img))  # Rescale to [0, 1]
            img = np.clip(img, 0, 1)  # Clip to ensure valid range for RGB

            img = (img * 255).astype(np.uint8)

            # Convert one-hot label to class index
            label = np.argmax(labels[i].numpy())
            plt.imshow(img)
            plt.title(f'Label: {label}')
            plt.axis('off')
            count += 1


    plt.tight_layout()
    plt.show()

In [ ]:
# Display augmented images from the training dataset
display_augmented_images(train_dataset, rows=4, cols=5)

##MODEL DEFINITION + TRAINING (+ eventual Fine Tuning) ⚙

In [ ]:
# Define the base model
base_model = InceptionResNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape
)
base_model.trainable = False  # Freeze the base model

# Define the model architecture
inputs = tfkl.Input(shape=input_shape)
x = base_model(inputs, training=False)
x = tfkl.GlobalAveragePooling2D()(x)
# Add Dense and Dropout layers
x = tfkl.Dense(256)(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Activation('relu')(x)
x = tfkl.Dropout(0.5)(x)
outputs = tfkl.Dense(num_classes, activation='softmax')(x) # Output layer with softmax activation

# Create the complete model
model = Model(inputs, outputs)

# Display the model architecture
model.summary()


In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',  # Suitable for one-hot encoded labels
    metrics=['accuracy']
)

In [ ]:
# Define Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)
#h5 only weights, .keras if whole model
model_checkpoint = ModelCheckpoint(
    'best_inceptionresnetv2_model.keras',  # Filename for the best model
    monitor='val_loss',
    save_best_only=True,
    mode='min'
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-6,
    verbose=1
)
# Combine all callbacks
callback_list = [early_stop, model_checkpoint, reduce_lr]

In [ ]:
print(f"Tensorflow version (2.17) -> {tf.__version__}")
print(f"Keras version (3.4.1) -> {tfk.__version__}")
print(f"Keras CV version -> {keras_cv.__version__}")

In [ ]:
# Train the model
epochs = 300  # Adjust this number based on your requirements
for batch in train_dataset.take(1): # Take one batch from the dataset
    print(f"Batch image shape: {batch[0].shape}")  # Print shape of images in the batch
    print(f"Batch label shape: {batch[1].shape}")  # Print shape of labels in the batch
    break # Stop after the first batch
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=val_dataset,
    callbacks=callback_list,
)

In [ ]:
# Plot training & validation accuracy and loss
plt.figure(figsize=(14, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.show()

##In case of loading a pretrained model: import + fine tuning


In [ ]:
# LOAD
best_model = tfk.models.load_model('best_inceptionresnetv2_model_087.keras')
print(f"Number of layers in the base model: {len(best_model.layers)}")

In [ ]:
print("Layers in best_model:")
for idx, layer in enumerate(best_model.layers):
    print(f"{idx}: {layer.name}")
best_model.summary()

In [ ]:
base_model = best_model.get_layer('inception_resnet_v2')  # Replace with the actual name
print(f"Number of layers in the base model: {len(base_model.layers)}")

In [ ]:
# Define Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)
#h5 only weights, .keras if whole model
model_checkpoint = ModelCheckpoint(
    'best_inceptionresnetv2_model.keras',  # Filename for the best model
    monitor='val_loss',
    save_best_only=True,
    mode='min'
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-6,
    verbose=1
)
best_model.summary()
# Combine all callbacks
callback_list = [early_stop, model_checkpoint, reduce_lr]

In [ ]:
from tensorflow.keras.optimizers import SGD
#FINE TUUNE
base_model.trainable = True

# Let's say we unfreeze the last 50 layers, by freezing everything but he last 50
for layer in base_model.layers[:-250]:
    layer.trainable = False


optimizer = SGD(learning_rate=1e-3, momentum=0.9, nesterov=True)
best_model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
# Fine-tuning with adjusted `initial_epoch`
fine_tune_epochs = 30  # Adjust as needed
total_epochs = fine_tune_epochs
history_fine = best_model.fit(
    train_dataset,
    epochs=total_epochs,
    initial_epoch=0,
    validation_data=val_dataset,
    callbacks=callback_list,
)


In [ ]:
# Plot training & validation accuracy and loss
plt.figure(figsize=(14, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history_fine.history['accuracy'], label='Train Accuracy')
plt.plot(history_fine.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history_fine.history['loss'], label='Train Loss')
plt.plot(history_fine.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.show()

In [ ]:
#when submitting, use this
best_model.optimizer = None
best_model.summary()
best_model.save('best_inceptionresnetv2_model__087_for_testing.keras', include_optimizer=False)


###if more needed

In [ ]:
# Optionally, re-freeze layers if you had unfrozen them earlier
base_model.trainable = True

for layer in base_model.layers[:-150]:
    layer.trainable = False

# Recompile the original model
best_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
fine_tune_epochs = 30  # Adjust as needed
last_epoch = history_fine.epoch[-1]
total_epochs = last_epoch + fine_tune_epochs + 1  # To continue from the next epoch

history_fine = best_model.fit(
    train_dataset,
    epochs=total_epochs,
    initial_epoch=last_epoch + 1,  # Start from the next epoch after initial training
    validation_data=val_dataset,
    callbacks=callback_list,
)

In [ ]:
# Plot training & validation accuracy and loss
plt.figure(figsize=(14, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history_fine.history['accuracy'], label='Train Accuracy')
plt.plot(history_fine.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history_fine.history['loss'], label='Train Loss')
plt.plot(history_fine.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.show()

###per class fine tuning, not sure if it works correctly (substitute output layer, train with subset of classes, then put the old output layer back)

In [ ]:
# Save the weights of the original output layer
original_output_layer = best_model.get_layer('dense_6')  # Replace 'dense' with your actual output layer name
original_weights = original_output_layer.get_weights()


In [ ]:
# Remove the existing output layer
x = best_model.layers[-2].output  # Assuming the second last layer is before output
new_output = Dense(4, activation='softmax', name='new_output')(x)
model_fine_tune = Model(inputs=best_model.input, outputs=new_output)

# Compile the fine-tuned model as before
model_fine_tune.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
classes_of_interest = [0, 3, 5, 6]

In [ ]:
# Create boolean masks for classes of interest
train_mask = np.isin(y_train_balanced, classes_of_interest)
val_mask = np.isin(y_val, classes_of_interest)

# Filter the training and validation data
X_train_fine = X_train_balanced[train_mask]
y_train_fine = y_train_balanced[train_mask]


X_val_fine = X_val[val_mask]
y_val_fine = y_val[val_mask]

In [ ]:
print(f"Filtered training samples: {X_train_fine.shape[0]}")
print(f"Filtered validation samples: {X_val_fine.shape[0]}")

In [ ]:
# Create a mapping from original labels to new labels
label_mapping = {original_label: new_label for new_label, original_label in enumerate(classes_of_interest)}

# Function to remap labels
def remap_labels(labels, mapping):
    return np.array([mapping[label] for label in labels])

# Remap the labels in training and validation data
y_train_fine_mapped = remap_labels(y_train_fine, label_mapping)
y_val_fine_mapped = remap_labels(y_val_fine, label_mapping)

# Verify the mapping
print("Original to New Label Mapping:")
for orig, new in label_mapping.items():
    print(f"Original Label {orig} -> New Label {new}")


In [ ]:
# Preprocessing function for training data
def preprocess_train3(images, labels):
   # Resize images
    images = tf.image.resize(images, [224, 224])

    # Normalize pixel values to [0, 1]
    images = tf.cast(images, tf.float32) / 255.0

    # Apply augmentations
    images = random_flip_layer(images)
    images = random_rotation_layer(images)
    images = random_zoom_layer(images)
    #images = grid_mask_layer(images)
    images = rand_augment_layer(images)

    # Rescale images back to [0, 255]
    images = images * 255.0
    # Apply InceptionResNetV2 preprocessing
    #expecting 0, 255
    images = preprocess_input(images)  # Scales to [-1, 1]

    # One-hot encode labels
    labels = tf.one_hot(tf.cast(labels, tf.int32), depth=4)

    return images, labels

# Preprocessing function for validation data
def preprocess_val3(images, labels):
    images = tf.image.resize(images, [224, 224])
    # Cast images to float32
    images = tf.cast(images, tf.float32)
    # Normalize and preprocess
    images = preprocess_input(images)

    labels = tf.one_hot(labels, depth=4)
    labels = tf.cast(labels, tf.float32)
    return images, labels

# Create TensorFlow Dataset from filtered training data
train_dataset_fine = tf.data.Dataset.from_tensor_slices((X_train_fine, y_train_fine_mapped))

# Shuffle, batch, and apply preprocessing
train_dataset_fine = train_dataset_fine.shuffle(buffer_size=1000).batch(batch_size)
train_dataset_fine = train_dataset_fine.map(
    preprocess_train3,
    num_parallel_calls=tf.data.AUTOTUNE
)
train_dataset_fine = train_dataset_fine.prefetch(tf.data.AUTOTUNE)

# Create TensorFlow Dataset from filtered validation data
val_dataset_fine = tf.data.Dataset.from_tensor_slices((X_val_fine, y_val_fine_mapped))

# Batch and apply preprocessing
val_dataset_fine = val_dataset_fine.batch(batch_size).map(
    preprocess_val3, num_parallel_calls=tf.data.AUTOTUNE
)
val_dataset_fine = val_dataset_fine.prefetch(tf.data.AUTOTUNE)


In [ ]:
model_layers = best_model.layers[:-1]
x = model_layers[-1].output

# Add new Dense layer with 4 classes
new_output = Dense(4, activation='softmax', name='new_output')(x)

# Create the new fine-tuned model
model_fine_tune = Model(inputs=best_model.input, outputs=new_output)

# Display the new model architecture
model_fine_tune.summary()

In [ ]:
# Freeze all layers in the base model
base_model.trainable = False

# Optionally, unfreeze the last N layers for fine-tuning
unfreeze_layers = 50  # Adjust based on experimentation and computational resources

for layer in base_model.layers[-unfreeze_layers:]:
    layer.trainable = True

print(f"Number of trainable layers in base_model after unfreezing last {unfreeze_layers} layers: "
      f"{sum([layer.trainable for layer in base_model.layers])}/{len(base_model.layers)}")


In [ ]:
# Compile the fine-tuned model
fine_tune_lr = 1e-5  # Lower learning rate for fine-tuning

model_fine_tune.compile(
    optimizer=Adam(learning_rate=fine_tune_lr),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Define training parameters
fine_tune_epochs = 10  # Adjust based on your needs
total_epochs = fine_tune_epochs  # If this is the initial fine-tuning phase

# Fine-tune the model
history_fine = model_fine_tune.fit(
    train_dataset_fine,
    epochs=total_epochs,
    validation_data=val_dataset_fine,
    callbacks=callback_list
)



In [ ]:
# Remove the fine-tuned output layer
model_fine_tune = Model(inputs=best_model.input, outputs=model_fine_tune.layers[-2].output)

# Add the original output layer back
original_output_layer_readded = Dense(8, activation='softmax', name='dense')(model_fine_tune.output)  # Replace 'dense' and 8 with your actual settings
best_model = Model(inputs=model_fine_tune.input, outputs=original_output_layer_readded)

# Load the original output layer weights
best_model.get_layer('dense').set_weights(original_weights)



# Now, best_model is back to its original state


In [ ]:
best_model.summary()

##TESTING ✊

In [ ]:
print(f"Tensorflow version (2.17) -> {tf.__version__}")
print(f"Keras version (3.4.1) -> {tfk.__version__}")

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Get true labels and predictions
y_true = np.concatenate([y for x, y in test_dataset], axis=0)
y_true = np.argmax(y_true, axis=1)

y_pred_probs = best_model.predict(test_dataset)
y_pred = np.argmax(y_pred_probs, axis=1)

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)
class_names = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7']

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred, target_names=class_names)
print(report)

In [ ]:
import matplotlib.pyplot as plt

# Get indices of misclassified and correctly classified samples
misclassified_indices = np.where(y_pred != y_true)[0]
correctly_classified_indices = np.where(y_pred == y_true)[0]

# Number of samples to display
num_samples = 5

# Display samples
fig, axes = plt.subplots(num_samples, 2, figsize=(10, 2 * num_samples))

for i in range(num_samples):
    # Misclassified samples
    if i < len(misclassified_indices):
        idx = misclassified_indices[i]
        image = X_test[idx]
        true_label = y_true[idx]
        predicted_label = y_pred[idx]
        axes[i, 0].imshow(image.astype('uint8'))
        axes[i, 0].set_title(f'Misclassified\nTrue: {class_names[true_label]}, Pred: {class_names[predicted_label]}')
        axes[i, 0].axis('off')
    else:
        axes[i, 0].axis('off')

    # Correctly classified samples
    if i < len(correctly_classified_indices):
        idx = correctly_classified_indices[i]
        image = X_test[idx]
        true_label = y_true[idx]
        predicted_label = y_pred[idx]
        axes[i, 1].imshow(image.astype('uint8'))
        axes[i, 1].set_title(f'Correctly Classified\nTrue: {class_names[true_label]}')
        axes[i, 1].axis('off')
    else:
        axes[i, 1].axis('off')

# Adjust layout
fig.tight_layout()
plt.show()


In [ ]:
probabilities = y_pred_probs

In [ ]:
import numpy as np  # Explicitly import numpy

# Index of the image to display probabilities for
# Use randint to get a random integer within the range
image_index = np.random.randint(0, len(train_dataset))

# Extract probabilities and display them
probs = probabilities[image_index]
for class_idx, prob in enumerate(probs):
    print(f'Class {class_idx} ({class_names[class_idx]}): {prob*100:.2f}%')

# Display the image alongside the probabilities
image = X_test[image_index]  # Assuming X_test contains the test images

plt.figure(figsize=(6, 6))
plt.imshow(image.astype('uint8'))
plt.axis('off')
plt.title(f"True Label: {class_names[y_true[image_index]]}\nPredicted: {class_names[np.argmax(probs)]}")
plt.show()



In [ ]:
plt.figure(figsize=(8, 4))
plt.bar(range(num_classes), probs)
plt.xticks(range(num_classes), class_names, rotation=45)
plt.ylabel('Probability')
plt.title('Predicted Probabilities')
plt.show()


##interpretability stuff

In [ ]:
print(f"Tensorflow version (2.17) -> {tf.__version__}")
print(f"Keras version (3.4.1) -> {tfk.__version__}")

###Grad CAM ? to try

In [ ]:
#needs last CONVOLUTIONAL LAYER!
base_model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random

# Define a function to recursively find a layer by name
def find_layer(model, layer_name):
    """
    Recursively search for a layer by name in a nested model.

    Args:
        model (tf.keras.Model): The model to search within.
        layer_name (str): The name of the layer to find.

    Returns:
        tf.keras.layers.Layer: The found layer.

    Raises:
        ValueError: If the layer is not found.
    """
    if model.name == layer_name:
        return model
    for layer in model.layers:
        if layer.name == layer_name:
            return layer
        elif isinstance(layer, Model):
            try:
                return find_layer(layer, layer_name)
            except ValueError:
                continue
    raise ValueError(f"Layer '{layer_name}' not found in the model.")

# Define a function to get the last Conv2D layer
def get_last_conv_layer(model):
    """
    Retrieves the last Conv2D layer in the model.

    Args:
        model (tf.keras.Model): The model to search within.

    Returns:
        tf.keras.layers.Layer: The last Conv2D layer.

    Raises:
        ValueError: If no Conv2D layer is found.
    """
    # Iterate through the layers in reverse order
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            return layer
        elif isinstance(layer, Model):
            try:
                return get_last_conv_layer(layer)
            except ValueError:
                continue
    raise ValueError("No Conv2D layer found in the model.")

# Define the Grad-CAM heatmap generation function with debugging
def make_gradcam_heatmap(img_array, model, last_conv_layer, pred_index=None):
    """
    Generates a Grad-CAM heatmap for a given image and model.

    Args:
        img_array (np.array): Preprocessed image array with shape (1, 224, 224, 3).
        model (tf.keras.Model): The model to analyze.
        last_conv_layer (tf.keras.layers.Layer): The last convolutional layer.
        pred_index (int, optional): Index of the class to generate heatmap for. Defaults to the top predicted class.

    Returns:
        np.array: Heatmap array.
    """
    try:
        # Create a model that maps the input image to the activations of the last conv layer and the output predictions
        grad_model = Model([model.inputs], [last_conv_layer.output, model.output])
        print("Grad model created successfully.")

        with tf.GradientTape() as tape:
            # Forward pass without specifying 'training' parameter
            conv_outputs, predictions = grad_model(img_array)
            print("Forward pass successful.")
            if pred_index is None:
                pred_index = tf.argmax(predictions[0])
                print(f"No pred_index provided, using top predicted class: {pred_index}")
            else:
                print(f"Using provided pred_index: {pred_index}")
            loss = predictions[:, pred_index]
            print(f"Loss shape: {loss.shape}")

        # Compute gradients of the loss with respect to the last conv layer output
        grads = tape.gradient(loss, conv_outputs)
        print(f"Gradients computed: {grads.shape}")

        # Compute the guided gradients
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2)).numpy()
        print(f"Pooled grads shape: {pooled_grads.shape}")

        # Multiply each channel in the feature map array by "how important this channel is" with regard to the top predicted class
        conv_outputs = conv_outputs[0].numpy()  # Shape: (H, W, C)
        print(f"Conv outputs shape: {conv_outputs.shape}")
        for i in range(pooled_grads.shape[0]):
            conv_outputs[:, :, i] *= pooled_grads[i]
        print("Applied pooled gradients to conv outputs.")

        # Compute the heatmap by averaging the feature map channels
        heatmap = np.mean(conv_outputs, axis=-1)
        print(f"Heatmap shape after mean: {heatmap.shape}")

        # Apply ReLU to the heatmap to keep only positive activations
        heatmap = np.maximum(heatmap, 0)
        print("Applied ReLU to heatmap.")

        # Normalize the heatmap
        max_heat = np.max(heatmap)
        if max_heat == 0:
            max_heat = 1e-10  # To prevent division by zero
        heatmap /= max_heat
        print("Normalized heatmap.")

        return heatmap
    except Exception as e:
        print("Exception in make_gradcam_heatmap:")
        print(e)
        raise

# Define the Grad-CAM display function
def display_gradcam(image, heatmap, class_names, y_true, y_pred, alpha=0.4):
    """
    Superimposes the Grad-CAM heatmap on the original image and displays it.

    Args:
        image (np.array): Original image array with shape (224, 224, 3).
        heatmap (np.array): Grad-CAM heatmap array.
        class_names (list): List of class names.
        y_true (int): True class index.
        y_pred (int): Predicted class index.
        alpha (float, optional): Transparency factor for heatmap. Defaults to 0.4.
    """
    try:
        # Resize heatmap to match image size
        heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
        print("Resized heatmap to match image size.")

        # Convert heatmap to RGB (apply color map)
        heatmap = np.uint8(255 * heatmap)
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        print("Applied color map to heatmap.")

        # Ensure image is in uint8
        if image.dtype != 'uint8':
            image = image.astype('uint8')
            print("Converted image to uint8.")

        # Superimpose the heatmap on the original image
        superimposed_img = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)
        print("Superimposed heatmap on image.")

        # Display the image
        plt.figure(figsize=(8, 8))
        plt.imshow(superimposed_img)
        plt.axis('off')
        plt.title(f"Grad-CAM\nTrue: {class_names[y_true]}, Pred: {class_names[y_pred]}")
        plt.show()
    except Exception as e:
        print("Exception in display_gradcam:")
        print(e)
        raise

# ----------------- Main Code Execution -----------------

# Define your class names
# Replace this with your actual class names
class_names = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7']

# Load your trained model
# Replace 'best_inceptionresnetv2_model.keras' with your actual model file
best_model = tf.keras.models.load_model('best_inceptionresnetv2_model.keras')
print("Model loaded successfully.")

# Verify model input shape
print(f"Model input shape: {best_model.input_shape}")

# Find the last Conv2D layer automatically
try:
    last_conv_layer = get_last_conv_layer(best_model)
    print(f"Last convolutional layer: {last_conv_layer.name}")
except ValueError as e:
    print(e)
    # Optionally, list available layers for debugging
    print("Available layers in the model:")
    for layer in best_model.layers:
        print(layer.name, layer.__class__.__name__)
    raise

# Check the shape of X_test
# Ensure that X_test is defined and loaded before this point
# Replace 'X_test' and 'y_test' with your actual test data variables
print(f"X_test shape: {X_test.shape}")

# Select a random image from the test set
image_index = random.randint(0, len(X_test) - 1)
image = X_test[image_index]
true_label = y_test[image_index]

# Prepare the image for the model
img = np.expand_dims(image, axis=0)  # Shape: (1, 96, 96, 3)

# Resize the image to match the model's expected input size
img_resized = tf.image.resize(img, [224, 224])  # Resize to 224x224

# Preprocess the image
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
img_processed = preprocess_input(img_resized)  # Ensure consistency with training preprocessing

# Verify image shape
print(f"Processed image shape ver: {img_processed.shape}")  # Should be (1, 224, 224, 3)

# Test standard prediction to ensure the model is working
try:
    predictions = best_model.predict(img_processed)
    predicted_class = np.argmax(predictions[0])
    print(f"Predicted class index: {predicted_class}")
except Exception as e:
    print("Error during model prediction:")
    print(e)
    raise

# Generate Grad-CAM heatmap
try:
    # Use 'predicted_class' directly
    heatmap = make_gradcam_heatmap(img_processed, best_model, last_conv_layer, pred_index=predicted_class)
    print(f"Heatmap shape: {heatmap.shape}")  # Should match resized image dimensions (224, 224)
    print(f"Heatmap max value: {np.max(heatmap)}")  # Should be 1.0 after normalization
except Exception as e:
    print("Error during Grad-CAM heatmap generation:")
    print(e)
    raise

# Display Grad-CAM
try:
    display_gradcam(image, heatmap, class_names, true_label, predicted_class)
except Exception as e:
    print("Error during Grad-CAM display:")
    print(e)
    raise


In [ ]:

# Select an image to visualize Grad-CAM
image_index = np.random.randint(0, len(X_test))
image = X_test[image_index]
true_label = y_true[image_index]
predicted_label = y_pred[image_index]

# Expand dimensions to match model input
img = np.expand_dims(image, axis=0)

# Resize to 299x299 as expected by InceptionResNetV2
#img_resized = tf.image.resize(img, [299, 299])

# Preprocess the image
img_processed = preprocess_val(image, labels)

# Create a model that maps the input image to the activations of the last conv layer and the model's output
grad_model = Model(
    inputs=base_model.inputs,
    outputs=[base_model.get_layer("conv_7b").output, best_model.output]
)
img_processed = preprocess_input(tf.image.resize(img, [224, 224]))
# Record operations for automatic differentiation
with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(img_processed)
    loss = predictions[:, predicted_label]

# Compute gradients of the loss with respect to the last conv layer output
grads = tape.gradient(loss, conv_outputs)

# Compute the guided gradients
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2)).numpy()

# Multiply each channel in the feature map array by "how important this channel is" with regard to the top predicted class
conv_outputs = conv_outputs[0].numpy()  # Shape: (H, W, C)
for i in range(pooled_grads.shape[0]):
    conv_outputs[:, :, i] *= pooled_grads[i]

# Compute the heatmap by averaging the feature map channels
heatmap = np.mean(conv_outputs, axis=-1)

# Apply ReLU to the heatmap to keep only positive activations
heatmap = np.maximum(heatmap, 0)

# Normalize the heatmap
heatmap /= np.max(heatmap) if np.max(heatmap) != 0 else 1

# Resize heatmap to match the original image size
heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))

# Convert heatmap to RGB (apply color map)
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Superimpose the heatmap on the original image
superimposed_img = cv2.addWeighted(image.astype('uint8'), 0.6, heatmap, 0.4, 0)

# Display the image
plt.figure(figsize=(8, 8))
plt.imshow(superimposed_img)
plt.axis('off')
plt.title(f"Grad-CAM\nTrue: {class_names[true_label]}, Pred: {class_names[predicted_label]}")
plt.show()


In [ ]:
img = X_test[image_index]
img = np.expand_dims(img, axis=0)


In [ ]:
# Preprocess the image as per your model's requirements
img_processed = preprocess_input(tf.image.resize(img, [224, 224]))


In [ ]:
grad_model = tfk.models.Model(
    [best_model.inputs],
    [best_model.get_layer('dense_6').output, best_model.output]
)


In [ ]:
with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(img_processed)
    predictions_tensor = predictions[0]
    loss = predictions_tensor[:, np.argmax(predictions_tensor[0])]

grads = tape.gradient(loss, conv_outputs)


In [ ]:
pooled_grads = tf.reduce_mean(grads, axis=(0))
conv_outputs = conv_outputs[0]
heatmap = tf.reshape(conv_outputs, (1, -1)) @ pooled_grads[..., tf.newaxis]

heatmap = tf.squeeze(heatmap)
heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)


In [ ]:
import cv2

img = img[0].astype('uint8')
heatmap = heatmap.numpy()
if heatmap.ndim == 0:
    print("0 dim")
    heatmap = heatmap.reshape(1, 1)
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()


###others

In [ ]:
 import shap

  # Create an explainer
  explainer = shap.GradientExplainer(model, X_train[:100])

  # Compute SHAP values
  shap_values = explainer.shap_values(X_val[:10])

  # Plot SHAP values for the first image
  shap.image_plot(shap_values, X_val[:10])

In [ ]:

from lime import lime_image
  from skimage.segmentation import mark_boundaries

  explainer = lime_image.LimeImageExplainer()
  explanation = explainer.explain_instance(X_val[0], model.predict, top_labels=5, hide_color=0, num_samples=1000)
  temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=10, hide_rest=False)
  plt.imshow(mark_boundaries(temp / 255.0, mask))
  plt.show()

